In [ ]:
!pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.2/491.2 kB 13.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 9.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 15.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 12.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 16.2 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.0
    Uninstalling fsspec-2025.3.0:
      Successfully uninstalled fsspec-2025.3.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cuda-cupti-cu12==12.4.127; platform_system =

In [5]:

!pip install datasets transformers torch scikit-learn pandas numpy nltk -q

from datasets import load_dataset, Dataset
import pandas as pd
import numpy as np
import re
import nltk
import torch
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, f1_score
from transformers import (
    AutoTokenizer,
    AutoModelForSequenceClassification,
    Trainer,
    TrainingArguments,
    pipeline
)


nltk.download('stopwords')


print("🛠️ Preparing dataset...")
dataset = load_dataset("yelp_review_full")

full_df = pd.DataFrame(dataset["train"].shuffle(seed=42).select(range(30000)))

def map_labels(star_rating):
    if star_rating <= 1: return 0
    elif star_rating == 2: return 1
    else: return 2

full_df["label"] = full_df["label"].apply(map_labels)

balanced_df = pd.concat([
    full_df[full_df["label"] == 0].sample(5000),
    full_df[full_df["label"] == 1].sample(5000),
    full_df[full_df["label"] == 2].sample(5000)
])

print("📊 Balanced distribution:\n", balanced_df["label"].value_counts())

print("\n🧹 Cleaning text...")
custom_stopwords = set(nltk.corpus.stopwords.words("english")) - {'not', 'no', 'nor', 'but'}

def clean_text(text):
    text = text.lower()
    text = re.sub(r"http\S+|@\w+|[^\w\s]|\d+", " ", text)
    text = re.sub(r"\s+", " ", text).strip()
    return " ".join([word for word in text.split() if word not in custom_stopwords])

balanced_df["cleaned_text"] = balanced_df["text"].apply(clean_text)


train_df, test_df = train_test_split(balanced_df, test_size=0.2, random_state=42)


print("\n🤖 Training LR Model...")
tfidf = TfidfVectorizer(
    max_features=10000,
    ngram_range=(1, 2),
    stop_words=list(custom_stopwords)
)

X_train_tf = tfidf.fit_transform(train_df["cleaned_text"])
X_test_tf = tfidf.transform(test_df["cleaned_text"])

lr_model = LogisticRegression(
    max_iter=1000,
    class_weight="balanced",
    solver="saga"
)
lr_model.fit(X_train_tf, train_df["label"])

print("\n📈 LR Performance:")
y_pred = lr_model.predict(X_test_tf)
print(classification_report(test_df["label"], y_pred))


print("\n🧠 Initializing Transformer...")
tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")
model = AutoModelForSequenceClassification.from_pretrained(
    "distilbert-base-uncased",
    num_labels=3,
    id2label={0: "negative", 1: "neutral", 2: "positive"}
)


hf_train = Dataset.from_pandas(train_df[["text", "label"]])
hf_test = Dataset.from_pandas(test_df[["text", "label"]])

def tokenize_function(examples):
    return tokenizer(
        examples["text"],
        padding="max_length",
        truncation=True,
        max_length=128,
        return_tensors="pt"
    )

tokenized_train = hf_train.map(tokenize_function, batched=True)
tokenized_test = hf_test.map(tokenize_function, batched=True)


def compute_metrics(p):
    preds = p.predictions.argmax(-1)
    f1_all = f1_score(p.label_ids, preds, average=None)
    return {
        "accuracy": (preds == p.label_ids).mean(),
        "f1_negative": f1_all[0],
        "f1_neutral": f1_all[1],
        "f1_positive": f1_all[2]
    }

training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=32,
    per_device_eval_batch_size=32,
    num_train_epochs=4,
    weight_decay=0.01,
    logging_steps=100,
    fp16=torch.cuda.is_available(),
    report_to="none",
    load_best_model_at_end=True,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_test,
    compute_metrics=compute_metrics,
)

print("\n🔥 Training Transformer...")
trainer.train()


class SentimentAnalyzer:
    def __init__(self):
        self.tfidf = tfidf
        self.lr_model = lr_model
        self.transformer_pipe = pipeline(
            "text-classification",
            model=model,
            tokenizer=tokenizer,
            device=0 if torch.cuda.is_available() else -1,
            top_k=3,
            truncation=True,
            max_length=128
        )

    def analyze(self, text):

        cleaned = clean_text(text)
        lr_pred = self.lr_model.predict(self.tfidf.transform([cleaned]))[0]


        tr_result = {res["label"]: res["score"] for res in self.transformer_pipe(text)[0]}

        return {
            "text": text,
            "traditional": ["negative", "neutral", "positive"][lr_pred],
            "transformer": max(tr_result, key=tr_result.get),
            "confidence": {k: f"{v:.2f}" for k, v in tr_result.items()}
        }


print("\n🔮 Testing Neutral Cases:")
analyzer = SentimentAnalyzer()

tests = [
    "The product works but nothing special really.",
    "Service was acceptable, though the wait time was too long.",
    "Food was average - not bad but not good either.",
    "Package arrived damaged but customer service resolved the issue.",
    "It's okay, I guess. Not worth the price but not terrible either."
]

for text in tests:
    result = analyzer.analyze(text)
    print(f"\n📝 {result['text']}")
    print(f"   Traditional: {result['traditional'].upper()}")
    print(f"   Transformer: {result['transformer'].upper()}")
    print(f"   Confidence: {result['confidence']}")

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 65.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 35.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 44.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 14.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 8.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 95.4 MB/s eta 0:00:00


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


🛠️ Preparing dataset...
📊 Balanced distribution:
 label
0    5000
1    5000
2    5000
Name: count, dtype: int64

🧹 Cleaning text...

🤖 Training LR Model...

📈 LR Performance:
              precision    recall  f1-score   support

           0       0.81      0.76      0.78      1037
           1       0.60      0.64      0.62       970
           2       0.75      0.74      0.75       993

    accuracy                           0.72      3000
   macro avg       0.72      0.71      0.72      3000
weighted avg       0.72      0.72      0.72      3000


🧠 Initializing Transformer...


Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Map:   0%|          | 0/12000 [00:00<?, ? examples/s]

...

/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1611: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(



🔥 Training Transformer...


Epoch,Training Loss,Validation Loss,Accuracy,F1 Negative,F1 Neutral,F1 Positive
1,0.687500,0.653895,0.717000,0.779736,0.604430,0.755725
2,0.575400,0.644587,0.727333,0.790871,0.620939,0.763845
3,0.458800,0.692681,0.717000,0.773650,0.604531,0.768024
4,0.371600,0.729125,0.711000,0.773603,0.596527,0.758901


Device set to use cuda:0



🔮 Testing Neutral Cases:

📝 The product works but nothing special really.
   Traditional: NEGATIVE
   Transformer: NEGATIVE
   Confidence: {'negative': '0.51', 'neutral': '0.47', 'positive': '0.02'}

📝 Service was acceptable, though the wait time was too long.
   Traditional: NEUTRAL
   Transformer: NEUTRAL
   Confidence: {'neutral': '0.66', 'negative': '0.28', 'positive': '0.06'}

📝 Food was average - not bad but not good either.
   Traditional: NEUTRAL
   Transformer: NEUTRAL
   Confidence: {'neutral': '0.59', 'negative': '0.39', 'positive': '0.02'}

📝 Package arrived damaged but customer service resolved the issue.
   Traditional: NEGATIVE
   Transformer: NEGATIVE
   Confidence: {'negative': '0.80', 'neutral': '0.17', 'positive': '0.03'}

📝 It's okay, I guess. Not worth the price but not terrible either.
   Traditional: NEGATIVE
   Transformer: NEUTRAL
   Confidence: {'neutral': '0.72', 'negative': '0.25', 'positive': '0.03'}
